In [10]:
import requests
import json
import time

# Server details
SERVER_IP = "10.0.0.11"
SERVER_PORT = "11451"
BASE_URL = "http://{}:{}".format(SERVER_IP, SERVER_PORT)

print("Test: add ipv4 label stack")
default_data = {
  "ipv4_src": "10.0.0.15/24 ",
  "ipv4_dst": "10.0.0.16/24",
  "tp": "tcp",
  "tp_src": "1001",
  "tp_dst": "1002",
  "label_list": "0x2502, 0x2507, 0x2505, 0x2509"
}
response = requests.post("{}/forward/ipv4/add".format(BASE_URL), json=default_data)
print(response.status_code, response.text)



Test: add ipv4 label stack
409 Already Exists


In [7]:
import requests
import networkx as nx
import matplotlib.pyplot as plt
import yaml
import sys
import os

FILE_INTERVAL = "\\" if os.name == "nt" else "/"
FILE_PATH = sys.path[0] + FILE_INTERVAL
CONFIG_PATH = sys.path[0] + FILE_INTERVAL + "config.py"

f = open("mapping.yaml", "r", encoding="utf-8")
content_2 = f.read()
parameter_2 = yaml.full_load(content_2)
f.close()


# 配置文件中的IP地址列表
SOUTHBOUND_SERVER_IPs = [
    '10.0.0.11','10.0.0.12','10.0.0.13'
]

# API端点
def fetch_topology_data(ip):
    """ 从API获取拓扑数据 """
    try:
        url = f"http://{ip}:11451/topology"
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            # print(ip_mapping[ip])
            print(ip, "连接成功")
            data = response.json()
            new_data = []
            for item in data:
                new_item = {}
                new_item['sw_ind'] = item['label']
                new_item['port'] = item['port'] 
                new_data.append(new_item)
            # 按 port 对 data 进行排序
            new_data.sort(key=lambda x: x['port'])
            print(new_data)
            return new_data
        else:
            print(f"无法连接到 {ip} - 状态码: {response.status_code}, 响应: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"无法连接到 {ip} - 错误: {e}")
        return None


data_list = [fetch_topology_data(ip) for ip in SOUTHBOUND_SERVER_IPs]

10.0.0.11 连接成功
[{'sw_ind': '0x2502', 'port': '128'}]
10.0.0.12 连接成功
[{'sw_ind': '0x2503', 'port': '132'}, {'sw_ind': '0x2501', 'port': '140'}]
10.0.0.13 连接成功
[{'sw_ind': '0x2502', 'port': '132'}]
